# Telecom X — Análisis de Evasión de Clientes (Churn)

_Notebook principal (EDA)._

In [ ]:
# !pip install pandas numpy matplotlib seaborn scikit-learn --quiet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option('display.max_columns', 100)
sns.set(style='whitegrid')


## 1) Carga de datos

In [ ]:
# Ruta local (ajusta el nombre del archivo)
DATA_PATH = Path('../data/telecom_churn.csv')

if DATA_PATH.exists():
    df = pd.read_csv(DATA_PATH)
else:
    # Ejemplo: si usas una URL pública, descomenta y reemplaza:
    # url = 'https://.../telecom_churn.csv'
    # df = pd.read_csv(url)
    raise FileNotFoundError(f'No se encontró {DATA_PATH}. Coloca el CSV en data/.')

df.head()

## 2) Inspección rápida

In [ ]:
print("Dimensiones:", df.shape)
display(df.sample(5, random_state=42))
display(df.info())
df.describe(include='all').T

## 3) Limpieza y transformaciones básicas

In [ ]:
# Estandarizar nombre de columna objetivo (churn)
posibles = [c for c in df.columns if c.lower() in ('churn','evasión','abandono','es_churn')]
if posibles:
    target_col = posibles[0]
else:
    target_col = 'churn'  # ajusta si tu dataset usa otro nombre

# Si la columna es Yes/No -> mapear a 1/0
if df[target_col].dtype == 'O':
    df[target_col] = df[target_col].astype(str).str.strip().str.lower().map({'yes':1,'si':1,'sí':1,'no':0}).fillna(df[target_col])
# Convertir 'TotalCharges' tipo texto a numérico (típico en datasets de churn)
if 'TotalCharges' in df.columns and df['TotalCharges'].dtype == 'O':
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'].replace(' ', np.nan), errors='coerce')

# Quitar duplicados simples
df = df.drop_duplicates().reset_index(drop=True)

# Nulos
nulos = df.isna().sum()
print("Nulos por columna:\n", nulos[nulos>0])


## 4) EDA — Visualizaciones

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))
rate = df[target_col].mean() if pd.api.types.is_numeric_dtype(df[target_col]) else df[target_col].map({1:1,0:0}).mean()
ax.bar(['Churn rate'], [rate*100])
ax.set_ylabel('%')
ax.set_title('Churn rate (%)')
plt.show()

In [ ]:
# Churn por variable categórica (ajusta alguna columna relevante si existe)
cat_cols = [c for c in df.select_dtypes('object').columns if c != target_col][:3]  # muestra hasta 3
for c in cat_cols:
    fig, ax = plt.subplots(figsize=(5,3))
    temp = df.groupby(c)[target_col].mean()*100
    temp.sort_values(ascending=False).plot(kind='bar', ax=ax)
    ax.set_title(f'Churn por {c} (%)')
    ax.set_ylabel('%')
    plt.tight_layout()
    plt.show()

In [ ]:
# Correlación (numéricas)
num = df.select_dtypes(include=[np.number])
if num.shape[1] >= 2:
    corr = num.corr()
    fig, ax = plt.subplots(figsize=(6,5))
    sns.heatmap(corr, cmap='RdBu_r', center=0, ax=ax, annot=False)
    ax.set_title('Matriz de correlación (numéricas)')
    plt.tight_layout()
    plt.show()

## 5) Insights (completa con tus hallazgos)
- ...
- ...
- ...